In [241]:
%matplotlib inline
import matplotlib.pyplot as plt
import numpy as np
import torch
from torch.autograd import Variable
import sys

def sigmoid(x):
    return 1 / (1 + math.exp(-x))
    
def sign(num):
    check = Variable(torch.Tensor([0.0]).double().expand(num.size()))
    return torch.ge(num,check).double()

def accuracy(y_real, y_predicted):
    n = y_real.size()[0]
    return (y_predicted.data==y_real.data).sum()/n

def predict(x,w1,w2):
    return torch.ge(torch.sigmoid(torch.sigmoid(x.mm(w1)).mm(w2)),0.5).double()

def accuracy2(y, y_hat):
    a=torch.sum(y.data==y_hat.data)
    n = y.size()[0]
    return a/n

In [210]:
images=np.load('data/images.npy')
labels=np.load('data/labels.npy')
labels=labels.astype(int)
labels[labels!=0]=-1
labels[labels==0]=1
labels[labels==-1]=0


In [211]:
shape_images_flat=(images.shape[0],images.shape[1]*images.shape[2])
images_flat=np.ndarray(shape=shape_images_flat)
for index in range(len(images)):
    images_flat[index]=images[index].flat
images_flat=(images_flat-images_flat.mean())/images.std()

In [212]:
train_images=images_flat[0:40000]
train_labels=labels[0:40000]

validation_images=images_flat[40000:45000]
validation_labels=labels[40000:45000]

test_images=images_flat[45000:50000]
test_labels=labels[45000:50000]

X = Variable(torch.from_numpy(images_flat)).double()
y = Variable(torch.from_numpy(labels).view(-1,1)).double()

In [216]:
def predict_nn(images,input_weights,input_bias_weight,hidden_weights,hidden_bias_weight):
    first_layer=torch.sigmoid(images.mm(input_weights)+input_bias_weight)
    second_layer=first_layer.mm(w2)+hidden_bias_weight
    return torch.ge(torch.sigmoid(second_layer),0.5).double()


In [173]:
training_accuracy_list=[]
validation_accuracy_list=[]
def compute_accuracy_graph(input_weights,input_bias_weight,hidden_weights,hidden_bias_weight):
    train_images_len=len(train_images)
    train_shuffler_list=list(range(0, train_images_len))
    random.shuffle(train_shuffler_list)    
    shuffled_train_images=train_images[train_shuffler_list]
    shuffled_train_labels=train_labels[train_shuffler_list]
    
    shuffled_train_images_used=shuffled_train_images[0:1000]
    shuffled_train_labels_used=shuffled_train_labels[0:1000]
    
    #d_train=shuffled_train_images_used.dot(W)
    d_train = predict_nn(shuffled_train_images_used,\
            input_weights,input_bias_weight,\
            hidden_weights,hidden_bias_weight)
    
    
    ac_train=accuracy_2(shuffled_train_labels_used,d_train)
    training_accuracy_list.append(ac_train)
    

    validation_images_len=len(validation_images)
    validation_shuffler_list=list(range(0, validation_images_len))
    random.shuffle(validation_shuffler_list)    
    shuffled_validation_images=validation_images[validation_shuffler_list]
    shuffled_validation_labels=validation_labels[validation_shuffler_list]  
    
    shuffled_validation_images_used=shuffled_validation_images[0:5000]
    shuffled_validation_labels_used=shuffled_validation_labels[0:5000]
    
    #d_validation=shuffled_validation_images_used.dot(W)
    
    d_validation = predict_nn(shuffled_validation_images_used,\
            input_weights,input_bias_weight,\
            hidden_weights,hidden_bias_weight)
    
    ac_validation=accuracy_2(shuffled_validation_labels_used,d_validation)
    validation_accuracy_list.append(ac_validation)
    
    

In [174]:
X_test = X[-5000:]
y_test = y[-5000:]

X_train = X[:40000]
X_dev = X[40000:45000]

y_train = y_shuffled[:40000]
y_dev = y_shuffled[40000:45000]



In [179]:
n,m = train_images.shape
hidden_layers = 35
output_nodes = 1
n=5000

In [180]:
train_accuracy = []

test_set_accuracy = []
epoch_points_train = []
epoch_points_dev = []
for super in range(1):
    w1 = Variable((torch.randn(m,hidden_layers)).double(),requires_grad=True) # m x P
    w2 = Variable((torch.randn(hidden_layers,output_nodes)).double(),requires_grad=True) # P x output
    
    m1_ = Variable((torch.zeros((m, hidden_layers)).double()))
    v1 = Variable((torch.zeros((m, hidden_layers)).double()))
    
    m2_ = Variable((torch.zeros((hidden_layers, output_nodes)).double()))
    v2 = Variable((torch.zeros((hidden_layers, output_nodes)).double()))
    t = 0
    for iterations in range(1):
        for i in range(n):
            
            mid_layer = torch.sigmoid(X_train[i].view(1,-1).mm(w1))
            out_layer = torch.sigmoid(mid_layer.mm(w2))
            
            eps1, eps2 = 1e-2,1e2
            loss = (y_train[i] * (out_layer.clamp(eps1, eps2).log())) + \
                ((1-y_train[i]) * ((1-out_layer).clamp(eps1, eps2).log()))  
            loss.backward()
 
            lr1 = learning_rate * w1.grad.data
            lr2 = learning_rate * w2.grad.data
            
            w1.data = w1.data + lr1
            w2.data = w2.data + lr2
            
            w2.grad.data.zero_()
            w1.grad.data.zero_()
            
            if i%5000 == 0:
                compute_accuracy_graph(w1.data.numpy(),0,w2.data.numpy(),0)
                train_accuracy.append(accuracy(y_train,predict(X_train, w1, w2)))
       
    a=accuracy(y_train,predict(X_train, w1, w2))
    print(a)
    

0.84935


In [242]:
a=accuracy(y_train,predict(X_train, w1, w2))
print(a)
weights=(w1,0,w2,0)
# a=X_train.data()
# b=y_train.data()
d_train = predict_nn(X_train,weights[0],weights[1],weights[2],weights[3])
ac=accuracy2(y_train,d_train)
print(ac)

0.8956
0.8956
